In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from datetime import datetime
import time
import os
import warnings
warnings.filterwarnings("ignore")

######################
# METADATA
######################
sim_start = datetime.now()
sims=1000
input_folder = r'C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\inputs'
input_file = 'inputs_v6.xlsx'
path = os.path.join(input_folder, input_file)

######################
# READ IN DATA
######################
sales_ci=pd.read_excel(path, sheet_name='sales_ci')
unc_pr=pd.read_excel(path, sheet_name='unc_pr')
unc_pr_desc=pd.read_excel(path, sheet_name='unc_pr_desc')
unc_pri=pd.read_excel(path, sheet_name='unc_pri')
unc_pri_desc=pd.read_excel(path, sheet_name='unc_pri_desc')

######################
# PREP BASELINE DATA
######################
sales_ci=sales_ci.drop_duplicates()
sales_ci=sales_ci.melt(id_vars=['product', 'region', 'units', 'indication'], 
        var_name="year", 
        value_name="sales")
sales_ci=sales_ci[(sales_ci['units'] =='¥')]

# Exclude indications that are not needed
exclude_ind = ['Adjustments', 'Central Adjustments', 'Total']
sales_ci=sales_ci[(~sales_ci['indication'].isin(exclude_ind))]

# Partition WW sales into 9 segments -> US, JP, CN, DE, FR, ES, IT, GB, WWex8
sales_ci_WW=sales_ci[(sales_ci['region']=='WW')]
ast8 = ['US','JP','CN', 'DE','FR','ES','IT','GB']
sales_ci_ast8=sales_ci[(sales_ci['region'].isin(ast8))]

sales_ci_WWex8=sales_ci_ast8.groupby(['product','units','indication','year']).sum().reset_index()
sales_ci_WWex8=sales_ci_WW.merge(sales_ci_WWex8, how='right', on=['product', 'units', 'indication', 'year'])
sales_ci_WWex8['sales']=sales_ci_WWex8['sales_x']-sales_ci_WWex8['sales_y']
sales_ci_WWex8['region']='WWex8'
sales_ci_WWex8 = sales_ci_WWex8[['product','region', 'units', 'indication', 'year', 'sales']]

# Generate new sales table
sales_ci_clean=pd.concat([sales_ci_ast8, sales_ci_WWex8])

# Add back non-strategic products
sales_ci_nsp=sales_ci[(sales_ci['indication']=='Non-Strategic')]
sales_ci_clean=pd.concat([sales_ci_clean, sales_ci_nsp])

# Create tag
sales_ci_clean['tag'] = sales_ci_clean['product'] + sales_ci_clean['region'] + sales_ci_clean['units'] + sales_ci_clean['indication'].astype(str)+ sales_ci_clean['year'].astype(str)

# Validation
print(sum(sales_ci_clean[(sales_ci_clean['year']==2025)]['sales']))  #18491
print(sum(sales_ci_clean[(sales_ci_clean['product']=='xtandi') & (sales_ci_clean['year']==2025)]['sales']))  #7309
print(sum(sales_ci_clean[(sales_ci_clean['product']=='mirabegron') & (sales_ci_clean['year']==2025)]['sales']))  #1223
print(sales_ci_clean['tag'].nunique()) # 920

# Remove interim dfs from memory
del sales_ci_WW
del sales_ci_ast8
del sales_ci_WWex8
del sales_ci_nsp
del sales_ci

# Save new sales data
sales_ci_clean.to_csv(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\output\sales_ci_clean.csv")

18491.327050511707
7309.662391952652
1223.22
920


In [2]:
%whos DataFrame

Variable         Type         Data/Info
---------------------------------------
sales_ci_clean   DataFrame                          pro<...>n\n[920 rows x 7 columns]
unc_pr           DataFrame          product region  yea<...>\n[484 rows x 34 columns]
unc_pr_desc      DataFrame       uncertainties         <...>        CN         0.50  
unc_pri          DataFrame        product region units <...>n\n[468 rows x 8 columns]
unc_pri_desc     DataFrame      uncertainties          <...>aint.  \n2  HSCT-Maint.  


# Phase 1: PTRS

## Simulation

In [3]:
#############
# CREATE OUTPUT DFS FOR PHASE 1
#############
output_p1 = pd.DataFrame(columns=['tag','product', 'region', 'units', 'indication', 'year', 'sales', 'sales_RA_10', 'sales_RA_25', 'sales_RA_50', 'sales_RA_75', 'sales_RA_90']) # Output of Phase 1
scenarios_p1 = pd.DataFrame(columns=['scenario', 'tag', 'product', 'region', 'units', 'indication', 'year', 'sales', 'sim_sales']) # Scenarios of Phase 1

#############
# GENERATE SCENARIOS
#############
# Add PTRS values to sales data
sales_ci_unc = sales_ci_clean.merge(unc_pri, how='right', on=['product','region', 'units', 'indication', 'year'])

scenario_sdf=[]
product_sdf=[]
region_sdf=[]
units_sdf=[]
ind_sdf=[]
year_sdf=[]
sales_sdf=[]
tag_sdf=[]
sim_sales_sdf=[]
    
# Loop through rows
for index, row in sales_ci_unc.iterrows():
# for index, row in sales_ci_unc.iloc[29:30].iterrows():
    n=sims
    sales_ra = []
    unc_ptrs_ra = []
    
    for i in range(n):
        scenario_sdf.append(i)
        product_sdf.append(row.values[0])
        region_sdf.append(row.values[1])
        units_sdf.append(row.values[2])
        ind_sdf.append(row.values[3])
        year_sdf.append(row.values[4])
        sales_sdf.append(row.values[5])
        tag_sdf.append(row.values[6])

        # Get sales (value)
        sales = row.values[5]

        # Get PTRS Uncertainty Probability (value)
        unc_ptrs_prob=np.random.binomial(size=1, n=1, p=row.values[7])
        
        # Get Uncertainty Value (value) for others
        unc1=row.values[8]
        unc2=row.values[9]
        
        if ((unc_ptrs_prob == 1) and ((~np.isnan(unc1)) or (~np.isnan(unc2)) )):
            m=sims
            for j in range(m):

                # Get Uncertainty Probability (value) for others
                unc1_prob=np.random.randint(1,4)
                
                if ((unc1_prob == 1)):
                    sales=unc1
                
                if ((unc1_prob == 2)):
                    sales=unc2
                    
        
        # Generate RA sales
        unc_all=float(sales)*float(unc_ptrs_prob)
        sales_ra.append(unc_all)
        sim_sales_sdf.append(unc_all)
        
    # Get Product Worldwide sales by year (series)
    prod_ww_sales = sales_ci_unc.iloc[index]
    prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
    prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
    prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
    prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
    prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
    prod_ww_sales['sales_P1_RA'] = np.mean(sales_ra)
    
    # Append to sales df
    output_p1 = output_p1.append(prod_ww_sales, ignore_index=True)
    
    # Develop scenario_df
    scenarios_p1= pd.DataFrame({'scenario': scenario_sdf,'tag': tag_sdf,'product': product_sdf,'region': region_sdf, 'units': units_sdf, 
                                'indication': ind_sdf, 'year': year_sdf, 'sales': sales_sdf, 'sim_sales': sim_sales_sdf})

# Save outputs
output_p1 = output_p1[['tag','product','region', 'units', 'indication', 'year', 'sales', 'sales_RA_10', 'sales_RA_25', 'sales_RA_50','sales_RA_75', 'sales_RA_90', 'sales_P1_RA']]

# Add non-simulated tags to output and scenario dfs
sales_exP1=sales_ci_clean[~(sales_ci_clean['tag'].isin(output_p1['tag']))]
sales_exP1['sales_RA_10']=sales_exP1['sales']
sales_exP1['sales_RA_25']=sales_exP1['sales']
sales_exP1['sales_RA_50']=sales_exP1['sales']
sales_exP1['sales_RA_75']=sales_exP1['sales']
sales_exP1['sales_RA_90']=sales_exP1['sales']
sales_exP1['sales_P1_RA']=sales_exP1['sales']
output_p1=pd.concat([output_p1, sales_exP1])

# Validation
print(sum(output_p1[(output_p1['year']==2025)]['sales']))  #18491
print(sum(output_p1[(output_p1['product']=='xtandi') & (output_p1['year']==2025)]['sales'])) #7309
print(sum(output_p1[(output_p1['product']=='mirabegron') & (output_p1['year']==2025)]['sales']))  #1223
print(output_p1['tag'].nunique()) # 920

18491.327050511714
7309.662391952652
1223.22
920


## Prep Results for Analysis

In [4]:
# Convert PTRS results to sales df for next phase (Commercial Uncertainty)
output_p1=output_p1[['tag','product','region', 'units', 'indication', 'year', 'sales', 'sales_P1_RA']]

# Sum sales at the WW level
output_p1_ww=output_p1.groupby(['product','units','indication','year']).sum().reset_index()
output_p1_ww['region']='Total'
output_p1=pd.concat([output_p1_ww, output_p1])

# Sum sales and scenarios at the Product level, as commercial uncertainty is applied at a higher granularity
output_p1_prod=output_p1.groupby(['product','units','region','year']).sum().reset_index()
output_p1_prod['indication']='Total'
output_p1=pd.concat([output_p1_prod, output_p1])

scenarios_p1_prod=scenarios_p1[scenarios_p1['product']=='fezo'].groupby(['product','units','region','year', 'scenario']).sum().reset_index()
scenarios_p1_prod['indication']='Total'
scenarios_p1=pd.concat([scenarios_p1_prod, scenarios_p1])

# Validation
print(sum(output_p1[(output_p1['year']==2025)& (output_p1['region']!='Total') & (output_p1['indication']!='Total')]['sales']))  #18491
print(sum(output_p1[(output_p1['product']=='xtandi') & (output_p1['year']==2025) & (output_p1['region']!='Total') & (output_p1['indication']!='Total')]['sales']))  #7309
print(sum(output_p1[(output_p1['product']=='mirabegron') & (output_p1['year']==2025) & (output_p1['region']!='Total') & (output_p1['indication']!='Total')]['sales']))  #1223
print(output_p1[(output_p1['region']!='Total') & (output_p1['indication']!='Total')]['tag'].nunique()) # 920

# Remove interim dfs from memory
del sales_exP1
del output_p1_ww
del output_p1_prod
del scenarios_p1_prod
del sales_ci_unc

output_p1.to_csv(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\output\output_p1.csv")
scenarios_p1.to_csv(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\output\scenarios_p1.csv")

18491.327050511714
7309.662391952652
1223.22
920


# Phase 2: Market Events / Commercial Uncertainty

## Prep for Phase 2

In [5]:
#############
# CREATE OUTPUT DFS FOR PHASE 2
#############
output_p2 = pd.DataFrame(columns=['tag','product', 'region', 'units', 'indication', 'year', 'sales', 'sales_RA_10', 'sales_RA_25','sales_RA_50','sales_RA_75', 'sales_RA_90']) # Outputs of Phase 2
scenarios_p2 = pd.DataFrame(columns=['scenario', 'tag', 'product', 'region', 'units', 'indication', 'year', 'sales', 'sim_sales']) # Scenarios of Phase 2
output_unc_p2 = pd.DataFrame(columns=['tag','product', 'region', 'units', 'indication', 'year', 'sales']) # Uncertainties of Phase 2

#############
# PREP INPUTS DFS FOR PHASE 2
#############
# Create input df
input_p2a=output_p1[(output_p1['region']=='Total') & (output_p1['indication']=='Total')] # These do not go thru phase2
input_p2b=output_p1[(output_p1['region']!='Total') & (output_p1['indication'].isin(['Total']))] # These go thru phase2

# Validation
print(sum(input_p2a[(input_p2a['year']==2025)]['sales']))  #18491
print(sum(input_p2a[(input_p2a['product']=='xtandi') & (input_p2a['year']==2025)]['sales']))  #7309
print(sum(input_p2a[(input_p2a['product']=='mirabegron') & (input_p2a['year']==2025)]['sales']))  #1223

print(sum(input_p2b[(input_p2b['year']==2025)]['sales']))  #18491
print(sum(input_p2b[(input_p2b['product']=='xtandi') & (input_p2b['year']==2025)]['sales']))  #7309
print(sum(input_p2b[(input_p2b['product']=='mirabegron') & (input_p2b['year']==2025)]['sales']))  #1223

18491.32705051171
7309.662391952652
1223.22
18491.3270505117
7309.66239195265
1223.2200000000003


## Simulation

In [6]:
#############
# XTANDI
#############
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB', 'WWex8']
product = 'xtandi'

# Clear output for rerun
output_p2=output_p2[(output_p2['product']!=product)]
output_unc_p2=output_unc_p2[(output_unc_p2['product']!=product)]

for year in years:
    print(year)
    
    for region in regions:
        n=sims
        sales_ra = []
        unc1_ra = []
        unc2_ra = []
        unc3_ra = []
        unc26_ra = []
        
        for i in range(n):
            # Get Uncertainty Probability (value)
            unc1_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc1')]['probability'], 1, 1)
            unc2_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc2')]['probability'], 1, 1)
            unc3_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc3')]['probability'], 1, 1)
            unc26_prob=np.random.binomial(size=1, n=1, p=unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc26')]['probability'])
            
            # Get Base Uncertainty
            unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_base']
            unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)
            
            # Get relevant sales values for uncertainties
            unc26_sales=input_p2b[(input_p2b['product']==product) & (input_p2b['year'] ==year) & (input_p2b['region']==region)]['sales']

            # Get Uncertainty Quant by year (series)
            unc1=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc1']
            unc2=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc2'])*float(unc26_sales)
            unc3=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc3'])*float(unc26_sales)
            unc26=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc26'])*float(unc26_sales)
            
            # Generate RA sales
            sales=input_p2b[(input_p2b['product'] ==product) & (input_p2b['region']==region) & (input_p2b['year']==year) ]
            unc_all=float(sales['sales_P1_RA']) + float(sales['sales_P1_RA']*unc_base_prob) + float(unc1*unc1_prob) + float(unc2*unc2_prob) + float(unc3*unc3_prob) + float(unc26*unc26_prob)
            sales_ra.append(unc_all)
            
            # Generate uncertainty arrays
            unc1_ra.append(float(unc1*unc1_prob))
            unc2_ra.append(float(unc2*unc2_prob))
            unc3_ra.append(float(unc3*unc3_prob))
            unc26_ra.append(float(unc26*unc26_prob))
            
        # Get Product Region sales by year (series)
        prod_ww_sales=input_p2b[(input_p2b['product'] ==product) & (input_p2b['region']==region) & (input_p2b['year']==year) ]
        prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
        prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
        prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
        prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
        prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
        prod_ww_sales['sales_P2_RA'] = np.mean(sales_ra)
        
        # Add uncertainty arrays to output_unc
        output_unc=input_p2b[(input_p2b['product'] ==product) & (input_p2b['region']==region) & (input_p2b['year']==year) ]
        output_unc['unc1_ra'] = np.mean(unc1_ra)
        output_unc['unc2_ra'] = np.mean(unc2_ra)
        output_unc['unc3_ra'] = np.mean(unc3_ra)
        output_unc['unc26_ra'] = np.mean(unc26_ra)
        # Append to sales df
        output_p2 = output_p2.append(prod_ww_sales, ignore_index=True)
        output_unc_p2 = output_unc_p2.append(output_unc, ignore_index=True)
    
output_p2=output_p2.fillna(0)
output_p2

2022
2023
2024
2025


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_P1_RA,sales_P2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,3421.620263,3459.604620,3509.620347,3558.548353,3593.378492,3490.227914,3509.515565
1,0,xtandi,JP,¥,Total,2022,438.632820,424.778881,429.039454,435.981325,442.505816,446.336399,438.632820,435.858913
2,0,xtandi,CN,¥,Total,2022,67.308338,65.276099,65.857505,66.932980,67.898080,68.421580,67.308338,66.883089
3,0,xtandi,DE,¥,Total,2022,518.145979,502.134944,506.901582,514.393512,522.171206,526.804844,518.145979,514.494771
4,0,xtandi,FR,¥,Total,2022,260.011082,251.910293,254.302546,258.228045,262.070812,264.429535,260.011082,258.192200
5,0,xtandi,IT,¥,Total,2022,127.421278,123.579168,124.668927,126.509512,128.277119,129.571178,127.421278,126.480674
6,0,xtandi,ES,¥,Total,2022,99.871451,96.809423,97.614613,98.987224,100.504816,101.506743,99.871451,99.103815
7,0,xtandi,GB,¥,Total,2022,116.406916,112.792087,113.957873,115.601108,117.392579,118.353344,116.406916,115.620592
8,0,xtandi,WWex8,¥,Total,2022,1087.663909,1053.669982,1064.925060,1080.488133,1096.051979,1106.042659,1087.663909,1080.331796
9,0,xtandi,US,¥,Total,2023,3892.422908,3828.105320,3872.474524,3936.460870,3995.694023,4035.094817,3892.422908,3933.965646


In [7]:
print(sum(output_p2[(output_p2['product'] =='xtandi')]['sales']))
print(sum(output_p2[(output_p2['product'] =='xtandi')]['sales_RA_25']))
print(sum(output_p2[(output_p2['product'] =='xtandi')]['sales_RA_50']))
print(sum(output_p2[(output_p2['product'] =='xtandi')]['sales_RA_75']))

26990.006769473104
26730.386444093445
27153.405559564915
27563.766619109618


In [8]:
#############
# XOSPATA
#############
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB', 'WWex8']
product = 'xospata'

# Clear output for rerun
output_p2=output_p2[(output_p2['product']!=product)]
output_unc_p2=output_unc_p2[(output_unc_p2['product']!=product)]


for year in years:
    print(year)
    
    for region in regions:
        n=sims
        sales_ra = []
        unc4_ra = []
        unc5_ra = []
        unc6_ra = []
        
        for i in range(n):
            # Get Uncertainty Probability (value)
            unc4_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc4')]['probability'], 1, 1)
            unc5_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc5')]['probability'], 1, 1)
            unc6_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc6')]['probability'], 1, 1)

            # Get Base Uncertainty
            unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_base']
            unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

            # Get relevant sales values for uncertainties
            unc4_sales=input_p2b[(input_p2b['product']==product) & (input_p2b['year'] ==year) & (input_p2b['region']==region)]['sales']
            unc5_sales=input_p2b[(input_p2b['product']==product) & (input_p2b['year'] ==year) & (input_p2b['region']==region)]['sales']
            unc6_sales=input_p2b[(input_p2b['product']==product) & (input_p2b['year'] ==year) & (input_p2b['region']==region)]['sales']
            
            # Get Uncertainty Quant by year (series)
            unc4=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc4'])*float(unc4_sales)
            unc5=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc5'])*float(unc5_sales)
            unc6=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc6'])*float(unc6_sales)

            # Generate RA sales
            sales=input_p2b[(input_p2b['product'] ==product) & (input_p2b['region']==region) & (input_p2b['year']==year) ]
            unc_all=float(sales['sales_P1_RA']) + float(sales['sales_P1_RA']*unc_base_prob) + float(unc4*unc4_prob) + float(unc5*unc5_prob) + float(unc6*unc6_prob)
            sales_ra.append(unc_all)
            
            # Generate uncertainty arrays
            unc4_ra.append(float(unc4*unc4_prob))
            unc5_ra.append(float(unc5*unc5_prob))
            unc6_ra.append(float(unc6*unc6_prob))
            
        # Get Product Region sales by year (series)
        prod_ww_sales=input_p2b[(input_p2b['product'] ==product) & (input_p2b['region']==region) & (input_p2b['year']==year) ]
        prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
        prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
        prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
        prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
        prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
        prod_ww_sales['sales_P2_RA'] = np.mean(sales_ra)
        
        # Add uncertainty arrays to output_unc
        output_unc=input_p2b[(input_p2b['product'] ==product) & (input_p2b['region']==region) & (input_p2b['year']==year) ]
        output_unc['unc4_ra'] = np.mean(unc4_ra)
        output_unc['unc5_ra'] = np.mean(unc5_ra)
        output_unc['unc6_ra'] = np.mean(unc6_ra)
        
        # Append to sales df
        output_p2 = output_p2.append(prod_ww_sales, ignore_index=True)
        output_unc_p2 = output_unc_p2.append(output_unc, ignore_index=True)
    
output_p2=output_p2.fillna(0)
output_p2

2022
2023
2024
2025


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_P1_RA,sales_P2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,3421.620263,3459.604620,3509.620347,3558.548353,3593.378492,3490.227914,3509.515565
1,0,xtandi,JP,¥,Total,2022,438.632820,424.778881,429.039454,435.981325,442.505816,446.336399,438.632820,435.858913
2,0,xtandi,CN,¥,Total,2022,67.308338,65.276099,65.857505,66.932980,67.898080,68.421580,67.308338,66.883089
3,0,xtandi,DE,¥,Total,2022,518.145979,502.134944,506.901582,514.393512,522.171206,526.804844,518.145979,514.494771
4,0,xtandi,FR,¥,Total,2022,260.011082,251.910293,254.302546,258.228045,262.070812,264.429535,260.011082,258.192200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0,xospata,FR,¥,Total,2025,10.287643,11.547057,11.740954,12.081867,12.407120,12.610657,12.626842,12.073548
68,0,xospata,IT,¥,Total,2025,41.269885,44.117821,44.987858,46.301667,47.381161,48.210078,48.425148,46.214190
69,0,xospata,ES,¥,Total,2025,9.511894,9.173141,9.326989,9.570041,9.828503,9.998175,10.097648,9.575708
70,0,xospata,GB,¥,Total,2025,13.035730,13.851275,14.101124,14.467485,14.839572,15.069720,15.200644,14.463819


In [9]:
print(sum(output_p2[(output_p2['product'] =='xospata')]['sales']))
print(sum(output_p2[(output_p2['product'] =='xospata')]['sales_RA_25']))
print(sum(output_p2[(output_p2['product'] =='xospata')]['sales_RA_50']))
print(sum(output_p2[(output_p2['product'] =='xospata')]['sales_RA_75']))   

2353.661500188844
2244.6802152273694
2305.25766606079
2363.4493676310753


In [10]:
#############
# PADCEV
#############
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB', 'WWex8']
product = 'padcev'

# Clear output for rerun
output_p2=output_p2[(output_p2['product']!=product)]
output_unc_p2=output_unc_p2[(output_unc_p2['product']!=product)]

for year in years:
    print(year)
    
    for region in regions:
        n=sims
        sales_ra = []
        unc7_ra = []
        unc8_ra = []
        unc9_ra = []
        unc10_ra = []
        unc11_ra = []
        unc12_ra = []
        unc13_ra = []
        unc14_ra = []
        
        for i in range(n):
            # Get Uncertainty Probability (value)
            unc7_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc7')]['probability'], 1, 1)
            unc8_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc8')]['probability'], 1, 1)
            unc9_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc9')]['probability'], 1, 1)
            unc10_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc10')]['probability'], 1, 1)
            unc11_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc11')]['probability'], 1, 1)
            unc12_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc12')]['probability'], 1, 1)
            unc13_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc13')]['probability'], 1, 1)
            unc14_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc14')]['probability'], 1, 1)
        
            # Get Base Uncertainty
            unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_base']
            unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

            # Get Uncertainty Quant by year (series)
            unc7=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc7']
            unc8=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc8']
            unc9=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc9']
            unc10=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc10']
            unc11=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc11']
            unc12=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc12']
            unc13=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc13']
            unc14=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc14']

            # Generate RA sales
            sales=input_p2b[(input_p2b['product'] ==product) & (input_p2b['region']==region) & (input_p2b['year']==year) ]
            unc_all=float(sales['sales_P1_RA']) + float(sales['sales_P1_RA']*unc_base_prob) + float(unc7*unc7_prob) + float(unc8*unc8_prob) + float(unc9*unc9_prob) + float(unc10*unc10_prob) + float(unc11*unc11_prob) + float(unc12*unc12_prob) + float(unc13*unc13_prob) + float(unc14*unc14_prob)
            sales_ra.append(unc_all)
            
            # Generate uncertainty arrays
            unc7_ra.append(float(unc7*unc7_prob))
            unc8_ra.append(float(unc8*unc8_prob))
            unc9_ra.append(float(unc9*unc9_prob))
            unc10_ra.append(float(unc10*unc10_prob))
            unc11_ra.append(float(unc11*unc11_prob))
            unc12_ra.append(float(unc12*unc12_prob))
            unc13_ra.append(float(unc13*unc13_prob))
            unc14_ra.append(float(unc14*unc14_prob))

        # Get Product Worldwide sales by year (series)
        prod_ww_sales=input_p2b[(input_p2b['product'] ==product) & (input_p2b['region']==region) & (input_p2b['year']==year) ]
        prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
        prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
        prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
        prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
        prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
        prod_ww_sales['sales_P2_RA'] = np.mean(sales_ra)
        
        # Add uncertainty arrays to output_unc
        output_unc=input_p2b[(input_p2b['product'] ==product) & (input_p2b['region']==region) & (input_p2b['year']==year) ]
        output_unc['unc7_ra'] = np.mean(unc7_ra)
        output_unc['unc8_ra'] = np.mean(unc8_ra)
        output_unc['unc9_ra'] = np.mean(unc9_ra)
        output_unc['unc10_ra'] = np.mean(unc10_ra)
        output_unc['unc11_ra'] = np.mean(unc11_ra)
        output_unc['unc12_ra'] = np.mean(unc12_ra)
        output_unc['unc13_ra'] = np.mean(unc13_ra)
        output_unc['unc14_ra'] = np.mean(unc14_ra)
    
        # Append to sales df
        output_p2 = output_p2.append(prod_ww_sales, ignore_index=True)
        output_unc_p2 = output_unc_p2.append(output_unc, ignore_index=True)
    
output_p2=output_p2.fillna(0)
output_p2

2022
2023
2024
2025


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_P1_RA,sales_P2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,3421.620263,3459.604620,3509.620347,3558.548353,3593.378492,3490.227914,3509.515565
1,0,xtandi,JP,¥,Total,2022,438.632820,424.778881,429.039454,435.981325,442.505816,446.336399,438.632820,435.858913
2,0,xtandi,CN,¥,Total,2022,67.308338,65.276099,65.857505,66.932980,67.898080,68.421580,67.308338,66.883089
3,0,xtandi,DE,¥,Total,2022,518.145979,502.134944,506.901582,514.393512,522.171206,526.804844,518.145979,514.494771
4,0,xtandi,FR,¥,Total,2022,260.011082,251.910293,254.302546,258.228045,262.070812,264.429535,260.011082,258.192200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,0,padcev,FR,¥,Total,2025,30.210385,12.302050,16.822758,21.592944,26.100767,29.479832,30.210385,21.299695
104,0,padcev,IT,¥,Total,2025,29.306934,25.733461,26.975826,29.228024,31.467056,32.729144,29.306934,29.214128
105,0,padcev,ES,¥,Total,2025,13.922966,12.225544,12.763316,13.865658,14.968886,15.640027,13.922966,13.884246
106,0,padcev,GB,¥,Total,2025,0.492994,0.433371,0.453486,0.490171,0.526593,0.550028,0.492994,0.490575


In [11]:
print(sum(output_p2[(output_p2['product'] =='padcev')]['sales']))
print(sum(output_p2[(output_p2['product'] =='padcev')]['sales_RA_25']))
print(sum(output_p2[(output_p2['product'] =='padcev')]['sales_RA_50']))
print(sum(output_p2[(output_p2['product'] =='padcev')]['sales_RA_75']))

4084.8039713788817
3830.382838394175
4170.171257447694
4502.953353752963


In [12]:
#############
# EVRENZO
#############
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB', 'WWex8']
product = 'evrenzo'

# Clear output for rerun
output_p2=output_p2[(output_p2['product']!=product)]
output_unc_p2=output_unc_p2[(output_unc_p2['product']!=product)]

for year in years:
    print(year)
    
    for region in regions:
        n=sims
        sales_ra = []
        unc15_ra = []
        unc16_ra = []
        unc17_ra = []
        unc18_ra = []
        
        for i in range(n):
            # Get Uncertainty Probability (value)
            unc15_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc15')]['probability'], 1, 1)
            unc16_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc16')]['probability'], 1, 1)
            unc17_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc17')]['probability'], 1, 1)
            unc18_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc18')]['probability'], 1, 1)
        
            # Get Base Uncertainty
            unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_base']
            unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

            # Get Uncertainty Quant by year (series)
            unc15=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc15']
            unc16=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc16']
            unc17=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc17']
            unc18=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc18']

            # Generate RA sales
            sales=input_p2b[(input_p2b['product'] ==product) & (input_p2b['region']==region) & (input_p2b['year']==year) ]
            unc_all=float(sales['sales_P1_RA']) + float(sales['sales_P1_RA']*unc_base_prob) + float(unc15*unc15_prob) + float(unc16*unc16_prob) + float(unc17*unc17_prob) + float(unc18*unc18_prob)
            sales_ra.append(unc_all)
            
            # Generate uncertainty arrays
            unc15_ra.append(float(unc15*unc15_prob))
            unc16_ra.append(float(unc16*unc16_prob))
            unc17_ra.append(float(unc17*unc17_prob))
            unc18_ra.append(float(unc18*unc18_prob))

        # Get Product Worldwide sales by year (series)
        prod_ww_sales=input_p2b[(input_p2b['product'] ==product) & (input_p2b['region']==region) & (input_p2b['year']==year) ]
        prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
        prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
        prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
        prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
        prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
        prod_ww_sales['sales_P2_RA'] = np.mean(sales_ra)
        
        # Add uncertainty arrays to output_unc
        output_unc=input_p2b[(input_p2b['product'] ==product) & (input_p2b['region']==region) & (input_p2b['year']==year) ]
        output_unc['unc15_ra'] = np.mean(unc15_ra)
        output_unc['unc16_ra'] = np.mean(unc16_ra)
        output_unc['unc17_ra'] = np.mean(unc17_ra)
        output_unc['unc18_ra'] = np.mean(unc18_ra)
        
        # Append to sales df
        output_p2 = output_p2.append(prod_ww_sales, ignore_index=True)
        output_unc_p2 = output_unc_p2.append(output_unc, ignore_index=True)
    
output_p2=output_p2.fillna(0)
output_p2

2022
2023
2024
2025


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_P1_RA,sales_P2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,3421.620263,3459.604620,3509.620347,3558.548353,3593.378492,3490.227914,3509.515565
1,0,xtandi,JP,¥,Total,2022,438.632820,424.778881,429.039454,435.981325,442.505816,446.336399,438.632820,435.858913
2,0,xtandi,CN,¥,Total,2022,67.308338,65.276099,65.857505,66.932980,67.898080,68.421580,67.308338,66.883089
3,0,xtandi,DE,¥,Total,2022,518.145979,502.134944,506.901582,514.393512,522.171206,526.804844,518.145979,514.494771
4,0,xtandi,FR,¥,Total,2022,260.011082,251.910293,254.302546,258.228045,262.070812,264.429535,260.011082,258.192200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,0,evrenzo,FR,¥,Total,2025,46.025024,39.038074,41.064126,44.837245,47.977887,50.240801,46.025024,44.629463
140,0,evrenzo,IT,¥,Total,2025,72.868764,45.784242,50.656084,56.768557,63.463741,69.114066,72.868764,57.061055
141,0,evrenzo,ES,¥,Total,2025,14.194413,10.411072,11.350782,12.406761,13.549753,14.463755,14.194413,12.407387
142,0,evrenzo,GB,¥,Total,2025,17.100499,15.094330,15.865719,17.155351,18.383635,19.140077,17.100499,17.114191


In [13]:
print(sum(output_p2[(output_p2['product'] =='evrenzo')]['sales']))
print(sum(output_p2[(output_p2['product'] =='evrenzo')]['sales_RA_25']))
print(sum(output_p2[(output_p2['product'] =='evrenzo')]['sales_RA_50']))
print(sum(output_p2[(output_p2['product'] =='evrenzo')]['sales_RA_75']))

1113.6799883146336
1013.4644782545282
1101.0407125651875
1188.3317502993277


In [14]:
#############
# ZOLBE
#############
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB', 'WWex8']
product = 'zolbe'

# Clear output for rerun
output_p2=output_p2[(output_p2['product']!=product)]
output_unc_p2=output_unc_p2[(output_unc_p2['product']!=product)]

for year in years:
    print(year)
    
    for region in regions:
        n=sims
        sales_ra = []
        
        for i in range(n):
            # Get Base Uncertainty
            unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_base']
            unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

            # Generate RA sales
            sales=input_p2b[(input_p2b['product'] ==product) & (input_p2b['region']==region) & (input_p2b['year']==year) ]
            unc_all=float(sales['sales_P1_RA']) + float(sales['sales_P1_RA']*unc_base_prob)
            sales_ra.append(unc_all)

        # Get Product Worldwide sales by year (series)
        prod_ww_sales=input_p2b[(input_p2b['product'] ==product) & (input_p2b['region']==region) & (input_p2b['year']==year) ]
        prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
        prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
        prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
        prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
        prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
        prod_ww_sales['sales_P2_RA'] = np.mean(sales_ra)
    
        # Append to sales df
        output_p2 = output_p2.append(prod_ww_sales, ignore_index=True)
    
output_p2=output_p2.fillna(0)
output_p2

2022
2023
2024
2025


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_P1_RA,sales_P2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,3421.620263,3459.604620,3509.620347,3558.548353,3593.378492,3490.227914,3509.515565
1,0,xtandi,JP,¥,Total,2022,438.632820,424.778881,429.039454,435.981325,442.505816,446.336399,438.632820,435.858913
2,0,xtandi,CN,¥,Total,2022,67.308338,65.276099,65.857505,66.932980,67.898080,68.421580,67.308338,66.883089
3,0,xtandi,DE,¥,Total,2022,518.145979,502.134944,506.901582,514.393512,522.171206,526.804844,518.145979,514.494771
4,0,xtandi,FR,¥,Total,2022,260.011082,251.910293,254.302546,258.228045,262.070812,264.429535,260.011082,258.192200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,0,zolbe,FR,¥,Total,2025,2.234815,1.974070,2.078688,2.251005,2.415129,2.511494,2.234815,2.243650
176,0,zolbe,IT,¥,Total,2025,3.637843,3.195058,3.354539,3.616347,3.911525,4.066398,3.637843,3.634278
177,0,zolbe,ES,¥,Total,2025,1.740246,1.529862,1.611862,1.738205,1.873961,1.953098,1.740246,1.741553
178,0,zolbe,GB,¥,Total,2025,2.423964,2.133499,2.237136,2.426056,2.606908,2.712507,2.423964,2.422216


In [15]:
print(sum(output_p2[(output_p2['product'] =='zolbe')]['sales']))
print(sum(output_p2[(output_p2['product'] =='zolbe')]['sales_RA_25']))
print(sum(output_p2[(output_p2['product'] =='zolbe')]['sales_RA_50']))
print(sum(output_p2[(output_p2['product'] =='zolbe')]['sales_RA_75']))  

475.20551843110303
438.9912679194326
474.97570366309395
510.53900437585


In [16]:
#############
# FEZO
#############
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB', 'WWex8']
product = 'fezo'

# Clear output for rerun
output_p2=output_p2[(output_p2['product']!=product)]
output_unc_p2=output_unc_p2[(output_unc_p2['product']!=product)]

for year in years:
    print(year)
    
    # Get Fezo US values, to create proportion
    fezo_base=output_p1[(output_p1['product']==product) & (output_p1['region']=='US') & (output_p1['year']==year)& (output_p1['indication']=='Total')]['sales_P1_RA']
    fezo_10 = float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region']=='US')]['unc19'])
    fezo_25 = float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region']=='US')]['unc20'])
    fezo_50 = float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region']=='US')]['unc21'])
    fezo_75 = float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region']=='US')]['unc22'])
    fezo_90 = float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region']=='US')]['unc23'])
    
    for region in regions:
        # Get Product Worldwide sales by year (series) 
        prod_ww_sales=input_p2b[(input_p2b['product'] ==product) & (input_p2b['region']==region) & (input_p2b['year']==year)]
        prod_ww_sales['sales_RA_10'] = float(fezo_10/fezo_base)*float(prod_ww_sales['sales_P1_RA'])
        prod_ww_sales['sales_RA_25'] = float(fezo_25/fezo_base)*float(prod_ww_sales['sales_P1_RA'])
        prod_ww_sales['sales_RA_50'] = float(fezo_50/fezo_base)*float(prod_ww_sales['sales_P1_RA'])
        prod_ww_sales['sales_RA_75'] = float(fezo_75/fezo_base)*float(prod_ww_sales['sales_P1_RA'])
        prod_ww_sales['sales_RA_90'] = float(fezo_90/fezo_base)*float(prod_ww_sales['sales_P1_RA'])
    
        prod_ww_sales['sales_P2_RA'] = prod_ww_sales['sales_RA_50']

        # Append to sales df
        output_p2 = output_p2.append(prod_ww_sales, ignore_index=True)

output_p2=output_p2.fillna(0)
output_p2

2022
2023
2024
2025


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_P1_RA,sales_P2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,3421.620263,3459.604620,3509.620347,3558.548353,3593.378492,3490.227914,3509.515565
1,0,xtandi,JP,¥,Total,2022,438.632820,424.778881,429.039454,435.981325,442.505816,446.336399,438.632820,435.858913
2,0,xtandi,CN,¥,Total,2022,67.308338,65.276099,65.857505,66.932980,67.898080,68.421580,67.308338,66.883089
3,0,xtandi,DE,¥,Total,2022,518.145979,502.134944,506.901582,514.393512,522.171206,526.804844,518.145979,514.494771
4,0,xtandi,FR,¥,Total,2022,260.011082,251.910293,254.302546,258.228045,262.070812,264.429535,260.011082,258.192200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,0,fezo,FR,¥,Total,2025,14.440805,7.631874,9.555493,12.287260,16.037749,19.651649,14.440805,12.287260
212,0,fezo,IT,¥,Total,2025,12.874491,6.804087,8.519062,10.954529,14.298223,17.520143,12.874491,10.954529
213,0,fezo,ES,¥,Total,2025,5.675226,2.999321,3.755302,4.828885,6.302824,7.723084,5.675226,4.828885
214,0,fezo,GB,¥,Total,2025,17.030000,9.000247,11.268766,14.490331,18.913271,23.175134,17.030000,14.490331


In [17]:
print(sum(output_p2[(output_p2['product'] =='fezo')]['sales']))
print(sum(output_p2[(output_p2['product'] =='fezo')]['sales_RA_25']))
print(sum(output_p2[(output_p2['product'] =='fezo')]['sales_RA_50']))
print(sum(output_p2[(output_p2['product'] =='fezo')]['sales_RA_75']))   

5336.288938830225
3531.050823911994
4540.364873198794
5926.499323240427


In [18]:
#############
# MIRABEGRON
#############
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB', 'WWex8']
product = 'mirabegron'

# Clear output for rerun
output_p2=output_p2[(output_p2['product']!=product)]

for region in regions:
    n=sims
    sales_ra22 = []
    sales_ra23 = []
    sales_ra24 = []
    sales_ra25 = []
    
    for i in range(n):
        # Get Base Uncertainty
        unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_base']
        unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

        # Get High,Base,Low Uncertainty
        unc_hbl_prob=np.random.randint(1,4)
        # unc_hbl_prob=1

        # Generate RA sales - High
        if (unc_hbl_prob==1):
            sales22=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==2022) & (unc_pr['region'] ==region)]['unc24'])*float(1+unc_base_prob)
            sales23=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==2023) & (unc_pr['region'] ==region)]['unc24'])*float(1+unc_base_prob)
            sales24=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==2024) & (unc_pr['region'] ==region)]['unc24'])*float(1+unc_base_prob)
            sales25=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==2025) & (unc_pr['region'] ==region)]['unc24'])*float(1+unc_base_prob)

        # Generate RA sales - Med
        if (unc_hbl_prob==2):
            sales22=float(input_p2b[(input_p2b['product'] ==product) & (input_p2b['region']==region) & (input_p2b['year']==2022) ]['sales'])*float(1+unc_base_prob)
            sales23=float(input_p2b[(input_p2b['product'] ==product) & (input_p2b['region']==region) & (input_p2b['year']==2023) ]['sales'])*float(1+unc_base_prob)
            sales24=float(input_p2b[(input_p2b['product'] ==product) & (input_p2b['region']==region) & (input_p2b['year']==2024) ]['sales'])*float(1+unc_base_prob)
            sales25=float(input_p2b[(input_p2b['product'] ==product) & (input_p2b['region']==region) & (input_p2b['year']==2025) ]['sales'])*float(1+unc_base_prob)
        
        # Generate RA sales - Low
        if (unc_hbl_prob==3):
            sales22=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==2022) & (unc_pr['region'] ==region)]['unc25'])*float(1+unc_base_prob)
            sales23=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==2023) & (unc_pr['region'] ==region)]['unc25'])*float(1+unc_base_prob)
            sales24=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==2024) & (unc_pr['region'] ==region)]['unc25'])*float(1+unc_base_prob)
            sales25=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==2025) & (unc_pr['region'] ==region)]['unc25'])*float(1+unc_base_prob)
        
        sales_ra22.append(sales22)
        sales_ra23.append(sales23)
        sales_ra24.append(sales24)
        sales_ra25.append(sales25)
 
     # Get Product, Region sales by year (series)
    prod_ww_sales=input_p2b[(input_p2b['product'] ==product) & (input_p2b['region']==region) & (input_p2b['year']==2022) ]
    prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra22, 10)
    prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra22, 25)
    prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra22, 50)
    prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra22, 75)
    prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra22, 90)
    prod_ww_sales['sales_P2_RA'] = np.mean(sales_ra22)

    # Append to sales df
    output_p2 = output_p2.append(prod_ww_sales, ignore_index=True)

    # Get Product Worldwide sales by year (series)
    prod_ww_sales=input_p2b[(input_p2b['product'] ==product) & (input_p2b['region']==region) & (input_p2b['year']==2023) ]
    prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra23, 10)
    prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra23, 25)
    prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra23, 50)
    prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra23, 75)
    prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra23, 90)
    prod_ww_sales['sales_P2_RA'] = np.mean(sales_ra23)

    # Append to sales df
    output_p2 = output_p2.append(prod_ww_sales, ignore_index=True)

    # Get Product Worldwide sales by year (series)
    prod_ww_sales=input_p2b[(input_p2b['product'] ==product) & (input_p2b['region']==region) & (input_p2b['year']==2024) ]
    prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra24, 10)
    prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra24, 25)
    prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra24, 50)
    prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra24, 75)
    prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra24, 90)
    prod_ww_sales['sales_P2_RA'] = np.mean(sales_ra24)

    # Append to sales df
    output_p2 = output_p2.append(prod_ww_sales, ignore_index=True)

    # Get Product Worldwide sales by year (series)
    prod_ww_sales=input_p2b[(input_p2b['product'] ==product) & (input_p2b['region']==region) & (input_p2b['year']==2025) ]
    prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra25, 10)
    prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra25, 25)
    prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra25, 50)
    prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra25, 75)
    prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra25, 90)
    prod_ww_sales['sales_P2_RA'] = np.mean(sales_ra25)

    # Append to sales df
    output_p2 = output_p2.append(prod_ww_sales, ignore_index=True)
    
output_p2=output_p2.fillna(0)
output_p2

,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_P1_RA,sales_P2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,3421.620263,3459.604620,3509.620347,3558.548353,3593.378492,3490.227914,3509.515565
1,0,xtandi,JP,¥,Total,2022,438.632820,424.778881,429.039454,435.981325,442.505816,446.336399,438.632820,435.858913
2,0,xtandi,CN,¥,Total,2022,67.308338,65.276099,65.857505,66.932980,67.898080,68.421580,67.308338,66.883089
3,0,xtandi,DE,¥,Total,2022,518.145979,502.134944,506.901582,514.393512,522.171206,526.804844,518.145979,514.494771
4,0,xtandi,FR,¥,Total,2022,260.011082,251.910293,254.302546,258.228045,262.070812,264.429535,260.011082,258.192200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,0,mirabegron,GB,¥,Total,2025,69.956730,67.098503,68.348659,69.954049,71.757125,72.810286,69.956730,69.994842
248,0,mirabegron,WWex8,¥,Total,2022,306.367270,293.743726,298.066390,306.003325,313.437775,318.455359,306.367270,306.065225
249,0,mirabegron,WWex8,¥,Total,2023,303.232806,287.588971,291.982506,299.509918,307.058838,311.206873,303.232806,299.411435
250,0,mirabegron,WWex8,¥,Total,2024,290.662682,275.456819,279.736840,286.921908,294.177195,298.153422,290.662682,286.854168


In [19]:
print(sum(output_p2[(output_p2['product'] =='mirabegron')]['sales']))
print(sum(output_p2[(output_p2['product'] =='mirabegron')]['sales_RA_25']))
print(sum(output_p2[(output_p2['product'] =='mirabegron')]['sales_RA_50']))
print(sum(output_p2[(output_p2['product'] =='mirabegron')]['sales_RA_75']))  

6512.1
5137.085652136624
6488.705485659742
7133.508704835576


In [20]:
#############
# TACROLIMUS
#############
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB', 'WWex8']
product = 'tacrolimus'

# Clear output for rerun
output_p2=output_p2[(output_p2['product']!=product)]

for year in years:
    print(year)
    
    for region in regions:
        n=sims
        sales_ra = []

        for i in range(n):
            # Get Base Uncertainty
            unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_base']
            unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)
            
            # Generate RA sales
            sales=input_p2b[(input_p2b['product'] ==product) & (input_p2b['region']==region) & (input_p2b['year']==year) ]
            unc_all=float(sales['sales_P1_RA']) + float(sales['sales_P1_RA']*unc_base_prob)
            sales_ra.append(unc_all)
            
        # Get Product Worldwide sales by year (series)
        prod_ww_sales=input_p2b[(input_p2b['product'] ==product) & (input_p2b['region']==region) & (input_p2b['year']==year) ]
        prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
        prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
        prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
        prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
        prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
        prod_ww_sales['sales_P2_RA'] = np.mean(sales_ra)
    
        # Append to sales df
        output_p2 = output_p2.append(prod_ww_sales, ignore_index=True)
    
output_p2=output_p2.fillna(0)
output_p2

2022
2023
2024
2025


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_P1_RA,sales_P2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,3421.620263,3459.604620,3509.620347,3558.548353,3593.378492,3490.227914,3509.515565
1,0,xtandi,JP,¥,Total,2022,438.632820,424.778881,429.039454,435.981325,442.505816,446.336399,438.632820,435.858913
2,0,xtandi,CN,¥,Total,2022,67.308338,65.276099,65.857505,66.932980,67.898080,68.421580,67.308338,66.883089
3,0,xtandi,DE,¥,Total,2022,518.145979,502.134944,506.901582,514.393512,522.171206,526.804844,518.145979,514.494771
4,0,xtandi,FR,¥,Total,2022,260.011082,251.910293,254.302546,258.228045,262.070812,264.429535,260.011082,258.192200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,0,tacrolimus,FR,¥,Total,2025,123.608011,118.607374,120.274150,123.724042,126.774733,128.597121,123.608011,123.614271
284,0,tacrolimus,IT,¥,Total,2025,36.814557,35.360471,35.898740,36.806020,37.828100,38.327428,36.814557,36.839985
285,0,tacrolimus,ES,¥,Total,2025,59.892065,57.407134,58.282550,59.784902,61.331208,62.233555,59.892065,59.828543
286,0,tacrolimus,GB,¥,Total,2025,54.303272,52.299704,53.036832,54.426236,55.698312,56.442520,54.303272,54.393010


In [21]:
print(sum(output_p2[(output_p2['product'] =='tacrolimus')]['sales']))
print(sum(output_p2[(output_p2['product'] =='tacrolimus')]['sales_RA_25']))
print(sum(output_p2[(output_p2['product'] =='tacrolimus')]['sales_RA_50']))
print(sum(output_p2[(output_p2['product'] =='tacrolimus')]['sales_RA_75']))  

6736.2300000000005
6567.887236781003
6736.9612197211645
6907.4058902148845


In [22]:
#############
# LEXISCAN
#############
years = [2022, 2023, 2024, 2025]
product = 'lexiscan'

# Clear output for rerun
output_p2=output_p2[(output_p2['product']!=product)]

for year in years:
    print(year)
    n=sims
    sales_ra = []
    
    for i in range(n):
        # Get Base Uncertainty
        unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year)& (unc_pr['region'] =='US')]['unc_base']
        unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

        # Generate RA sales
        sales=input_p2b[(input_p2b['product'] ==product) & (input_p2b['year']==year) ]
        unc_all=float(sales['sales_P1_RA']) + float(sales['sales_P1_RA']*unc_base_prob)
        sales_ra.append(unc_all)

    # Get Product Worldwide sales by year (series)
    prod_ww_sales=input_p2b[(input_p2b['product'] ==product) & (input_p2b['year']==year) ]
    prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
    prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
    prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
    prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
    prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
    prod_ww_sales['sales_P2_RA'] = np.mean(sales_ra)

    
    # Append to sales df
    output_p2 = output_p2.append(prod_ww_sales, ignore_index=True)
    
output_p2=output_p2.fillna(0)
output_p2

2022
2023
2024
2025


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_P1_RA,sales_P2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,3421.620263,3459.604620,3509.620347,3558.548353,3593.378492,3490.227914,3509.515565
1,0,xtandi,JP,¥,Total,2022,438.632820,424.778881,429.039454,435.981325,442.505816,446.336399,438.632820,435.858913
2,0,xtandi,CN,¥,Total,2022,67.308338,65.276099,65.857505,66.932980,67.898080,68.421580,67.308338,66.883089
3,0,xtandi,DE,¥,Total,2022,518.145979,502.134944,506.901582,514.393512,522.171206,526.804844,518.145979,514.494771
4,0,xtandi,FR,¥,Total,2022,260.011082,251.910293,254.302546,258.228045,262.070812,264.429535,260.011082,258.192200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,0,tacrolimus,WWex8,¥,Total,2025,706.677430,677.881066,688.406071,706.525320,725.325819,735.720548,706.677430,706.833044
288,0,lexiscan,WW,¥,Total,2022,759.733936,729.156563,741.127635,759.227429,777.271284,788.562752,759.733936,758.985209
289,0,lexiscan,WW,¥,Total,2023,379.866968,364.631690,370.182811,379.296557,388.556101,394.031959,379.866968,379.459249
290,0,lexiscan,WW,¥,Total,2024,18.993348,18.268751,18.553254,19.017413,19.505282,19.784858,18.993348,19.022857


In [23]:
print(sum(output_p2[(output_p2['product'] =='lexiscan')]['sales']))
print(sum(output_p2[(output_p2['product'] =='lexiscan')]['sales_RA_25']))
print(sum(output_p2[(output_p2['product'] =='lexiscan')]['sales_RA_50']))
print(sum(output_p2[(output_p2['product'] =='lexiscan')]['sales_RA_75']))

1174.7385985710891
1145.6039927274007
1173.6724217819065
1201.8553913427031


In [24]:
#############
# OTHERS
#############
years = [2022, 2023, 2024, 2025]
region = 'WW'
other_products = ['evenity','cresemba','tamsulosin','suglat','ambisome','vesicare','mycamine','sujanu','symraf','cimzia','blincyto','repatha','linzess',
'vesomni','myslee/stilnox','irribow','gonax','feburic','asamax','josamycin','non prod','istodax','other merchandise products','allelock','tr','acr','blz','bonoteo','seroquel','geninax','col','inf-v','kiklin',
'ofa','regnite','srs','st','dificlir','p_aco - acofide','other astellas products']

for product in other_products:
    # Clear output for rerun
    output_p2=output_p2[(output_p2['product']!=product)]
    print(product)
    for year in years:
        n=sims
        sales_ra = []
    
        for i in range(n):
            # Get Base Uncertainty
            unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year)]['unc_base']
            unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

            # Generate RA sales
            sales=input_p2b[(input_p2b['product'] ==product) & (input_p2b['region']==region) & (input_p2b['year']==year) ]
            unc_all=float(sales['sales_P1_RA']) + float(sales['sales_P1_RA']*unc_base_prob)
            sales_ra.append(unc_all)

        # Get Product Worldwide sales by year (series)
        prod_ww_sales=input_p2b[(input_p2b['product'] ==product) & (input_p2b['region']==region) & (input_p2b['year']==year) ]
        prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
        prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
        prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
        prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
        prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
        prod_ww_sales['sales_P2_RA'] = np.mean(sales_ra)
    

        # Append to sales df
        output_p2 = output_p2.append(prod_ww_sales, ignore_index=True)
    
output_p2=output_p2.fillna(0)
output_p2

evenity
cresemba
tamsulosin
suglat
ambisome
vesicare
mycamine
sujanu
symraf
cimzia
blincyto
repatha
linzess
vesomni
myslee/stilnox
irribow
gonax
feburic
asamax
josamycin
non prod
istodax
other merchandise products
allelock
tr
acr
blz
bonoteo
seroquel
geninax
col
inf-v
kiklin
ofa
regnite
srs
st
dificlir
p_aco - acofide
other astellas products


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_P1_RA,sales_P2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,3421.620263,3459.604620,3509.620347,3558.548353,3593.378492,3490.227914,3509.515565
1,0,xtandi,JP,¥,Total,2022,438.632820,424.778881,429.039454,435.981325,442.505816,446.336399,438.632820,435.858913
2,0,xtandi,CN,¥,Total,2022,67.308338,65.276099,65.857505,66.932980,67.898080,68.421580,67.308338,66.883089
3,0,xtandi,DE,¥,Total,2022,518.145979,502.134944,506.901582,514.393512,522.171206,526.804844,518.145979,514.494771
4,0,xtandi,FR,¥,Total,2022,260.011082,251.910293,254.302546,258.228045,262.070812,264.429535,260.011082,258.192200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,0,p_aco - acofide,WW,¥,Total,2025,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
448,0,other astellas products,WW,¥,Total,2022,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
449,0,other astellas products,WW,¥,Total,2023,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
450,0,other astellas products,WW,¥,Total,2024,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [25]:
print(sum(output_p2[(output_p2['product'].isin(other_products))]['sales']))
print(sum(output_p2[(output_p2['product'].isin(other_products))]['sales_RA_25']))
print(sum(output_p2[(output_p2['product'].isin(other_products))]['sales_RA_50']))
print(sum(output_p2[(output_p2['product'].isin(other_products))]['sales_RA_75']))

9797.389256678245
9305.687039015535
9798.260194558208
10288.565842726206


## Prep Results for Analysis

In [26]:
# Add WW sales to output
output_p2a=output_p2
output_p2a=output_p2a.groupby(['product','units','indication','year']).sum().reset_index()
output_p2a['region']='ALL'
output_p2 = output_p2.append(output_p2a, ignore_index=True)

# Consolidate uncertainty output
output_unc_p2=output_unc_p2.groupby(['product','units','indication','year', 'region']).max().reset_index()

# Remove interim dfs from memory
del input_p2a 
del input_p2b 
del sales
del prod_ww_sales
del output_unc
del output_p2a

output_p2.to_csv(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\output\output_p2.csv")
output_unc_p2.to_csv(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\output\output_unc_p2.csv")
scenarios_p2.to_csv(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\output\scenarios_p2.csv")

In [27]:
%whos DataFrame

Variable         Type         Data/Info
---------------------------------------
output_p1        DataFrame                          pro<...>\n[1816 rows x 8 columns]
output_p2        DataFrame         tag product region u<...>\n[648 rows x 14 columns]
output_unc_p2    DataFrame         product units indica<...>\n[144 rows x 27 columns]
sales_ci_clean   DataFrame                          pro<...>n\n[920 rows x 7 columns]
scenarios_p1     DataFrame           product units regi<...>[504000 rows x 9 columns]
scenarios_p2     DataFrame    Empty DataFrame\nColumns:<...>es, sim_sales]\nIndex: []
unc_pr           DataFrame          product region  yea<...>\n[484 rows x 34 columns]
unc_pr_desc      DataFrame       uncertainties         <...>        CN         0.50  
unc_pri          DataFrame        product region units <...>n\n[468 rows x 8 columns]
unc_pri_desc     DataFrame      uncertainties          <...>aint.  \n2  HSCT-Maint.  


# Phase 3: Above Brand

## Prep for Phase 3 and Simulation

In [ ]:
# Create output dfs
output_p3 = pd.DataFrame(columns=['tag','product', 'region', 'units', 'indication', 'year', 'sales', 'sales_RA_10', 'sales_RA_25','sales_RA_50','sales_RA_75', 'sales_RA_90', 
                                  'sales_P1_RA', 'sales_P2_RA', 'sales_P3_RA']) # Outputs of Phase 3

scenarios_p3 = pd.DataFrame(columns=['scenario', 'tag', 'product', 'region', 'units', 'indication', 'year', 'sales', 'sim_sales']) # Scenarios of Phase 3
output_unc_p3 = pd.DataFrame(columns=['tag','product', 'region', 'units', 'indication', 'year', 'sales', 'variable', 'value']) # Uncertainties of Phase 3

# Inputs
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB']
products = ['evrenzo', 'fezo', 'mirabegron', 'padcev', 'tacrolimus', 'xospata', 'xtandi', 'zolbe']
input_p3a=output_p2[~(output_p2['region'].isin(regions))] # These do not go thru phase3
input_p3b=output_p2[(output_p2['region'].isin(regions))] # These go thru phase3

for product in products:
    print(product)

    for year in years:
    
        for region in regions:
            n=sims
            sales_ra = []
            unc_ab1_ra = []
            unc_ab2_ra = []
            unc_ab3_ra = []
            unc_ab4_ra = []

            for i in range(n):
                # Get Above Brand Uncertainty Probability (value)
                unc_ab1_prob=np.random.binomial(size=1, n=1, p=unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc_ab1')]['probability'])
                unc_ab2_prob=np.random.binomial(size=1, n=1, p=unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc_ab2')]['probability'])
                unc_ab3_prob=np.random.binomial(size=1, n=1, p=unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc_ab3')]['probability'])
                unc_ab4_prob=np.random.binomial(size=1, n=1, p=unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc_ab4')]['probability'])
                
                # Get sales from phase 2
                sales=input_p3b[(input_p3b['product'] ==product) & (input_p3b['region']==region) & (input_p3b['year']==year) ]
                
                # Get Uncertainty Quant by year (series)
                unc_ab1=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_ab1']*float(sales['sales_P2_RA'])*unc_ab1_prob
                unc_ab2=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_ab2']*float(sales['sales_P2_RA'])*unc_ab2_prob
                unc_ab3=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_ab3']*float(sales['sales_P2_RA'])*unc_ab3_prob
                unc_ab4=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_ab4']*float(sales['sales_P2_RA'])*unc_ab4_prob
                
                # Generate RA sales
                unc_all=float(sales['sales_P2_RA']) + unc_ab1 + unc_ab2 + unc_ab3 + unc_ab4
                sales_ra.append(unc_all)
                
                # Generate uncertainty arrays
                unc_ab1_ra.append(float(unc_ab1))
                unc_ab2_ra.append(float(unc_ab2))
                unc_ab3_ra.append(float(unc_ab3))
                unc_ab4_ra.append(float(unc_ab4))
            
            # Get Product Worldwide sales by year (series)
            prod_ww_sales=input_p3b[(input_p3b['product'] ==product) & (input_p3b['region']==region) & (input_p3b['year']==year) ]
            prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
            prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
            prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
            prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
            prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
            prod_ww_sales['sales_P3_RA'] = np.mean(sales_ra)
            
            # Add uncertainty arrays to output_unc
            output_unc=input_p3b[(input_p3b['product'] ==product) & (input_p3b['region']==region) & (input_p3b['year']==year) ][['tag', 'product', 'region', 'units', 'indication', 'year', 'sales','sales_P1_RA', 'sales_P2_RA']]
            output_unc['unc_ab1_ra'] = np.mean(unc_ab1_ra)
            output_unc['unc_ab2_ra'] = np.mean(unc_ab2_ra)
            output_unc['unc_ab3_ra'] = np.mean(unc_ab3_ra)
            output_unc['unc_ab4_ra'] = np.mean(unc_ab4_ra)

            # Append to sales df
            output_p3 = output_p3.append(prod_ww_sales, ignore_index=True)
            output_unc_p2 = output_unc_p2.append(output_unc, ignore_index=True)
            
output_p3=output_p3.fillna(0)
output_p3

evrenzo
fezo
mirabegron
padcev
tacrolimus
xospata


## Prep Results for Analysis

In [ ]:
# Update output uncertainty df for PTRS
output_unc_p2=output_unc_p2.groupby(['product','units','indication','year', 'region']).max().reset_index()
output_unc_p2['unc_ptrs'] = output_unc_p2['sales_P1_RA'] - output_unc_p2['sales']

# Update output uncertainty df for PTRS
output_unc_p2=output_unc_p2.fillna(0)

# Select columns needed
output_unc_p3=output_unc_p2[['tag', 'product', 'region', 'units', 'indication', 'year', 'sales',
                             'unc_ptrs','unc1_ra', 'unc2_ra', 'unc3_ra', 'unc4_ra',
                             'unc5_ra', 'unc6_ra', 'unc7_ra', 'unc8_ra', 'unc9_ra', 'unc10_ra',
                             'unc11_ra', 'unc12_ra', 'unc13_ra', 'unc14_ra', 'unc15_ra', 'unc16_ra',
                             'unc17_ra', 'unc18_ra', 'unc26_ra', 
                             'unc_ab1_ra', 'unc_ab2_ra','unc_ab3_ra', 'unc_ab4_ra']]

# Pivot table
id_vars=['tag', 'product', 'region', 'units', 'indication', 'year', 'sales']
value_vars=['unc_ptrs','unc1_ra', 'unc2_ra', 'unc3_ra', 'unc4_ra',
            'unc5_ra', 'unc6_ra', 'unc7_ra', 'unc8_ra', 'unc9_ra', 'unc10_ra',
            'unc11_ra', 'unc12_ra', 'unc13_ra', 'unc14_ra', 'unc15_ra', 'unc16_ra',
            'unc17_ra', 'unc18_ra', 'unc26_ra', 
            'unc_ab1_ra', 'unc_ab2_ra','unc_ab3_ra', 'unc_ab4_ra']

output_unc_p3=pd.melt(output_unc_p3, id_vars=id_vars, value_vars=value_vars)

# # Merge uncertainty name to output_unc_p22
# output_unc_p22=output_unc_p22.merge(unc_pr_desc, how='left', left_on='lkey', right_on='rkey')

# Add back tags that did not go through phase 3
input_p3a['sales_P3_RA']=input_p3a['sales_P2_RA']
output_p3=pd.concat([output_p3, input_p3a[input_p3a['region']!='ALL']], ignore_index=True, axis=0)

# Add WW sales to output
output_phase3_agg=output_p3
output_phase3_agg=output_phase3_agg.groupby(['product','units','indication','year']).sum().reset_index()
output_phase3_agg['region']='ALL'
output_p3 = output_p3.append(output_phase3_agg, ignore_index=True)

# Remove interim dfs from memory
del prod_ww_sales
del sales
del input_p3a 
del input_p3b 
del output_phase3_agg

output_p3.to_csv(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\output\output_p3.csv")
output_unc_p3.to_csv(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\output\output_unc_p3.csv")
scenarios_p3.to_csv(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\output\scenarios_p3.csv")

In [ ]:
%whos DataFrame

# Metadata

In [ ]:
sim_end = datetime.now()
td =  sim_end - sim_start
td_mins = int(round(td.total_seconds() / 60))
metatdata = {'sim_start': sim_start, 'sim_end': sim_end, 'sims_run': sims, 'sim_time': td_mins}
metatdata_df=pd.DataFrame.from_dict(metatdata, orient='index')
metatdata_df.to_csv(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\output\metatdata.csv")